In [2]:
from gensim.models import KeyedVectors

C:\Users\abybi\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
glove_path = '../Downloads/glove.6B/glove.6B.100d.txt'

In [4]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each lengh 200
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-50")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

In [5]:
wv_from_bin = load_embedding_model()


Loaded vocab size 400000


In [6]:
def map_words_to_vector(text, model):
    words = text.split()
    vectors = []
    for word in words:
        if word in model:
            vectors.append(model[word])
    return np.array(vectors)

In [7]:
import numpy as np
import pandas as pd
df = pd.read_csv('IMDB Dataset.csv')

In [8]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [10]:
df.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [11]:
df.sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [12]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize,WordNetLemmatizer
nltk.download('all')
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping grammars\basque_grammars.zip.
[nltk_data]   

[nltk_data]    | Downloading package nonbreaking_prefixes to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nonbreaking_prefixes.zip.
[nltk_data]    | Downloading package nps_chat to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\nps_chat.zip.
[nltk_data]    | Downloading package omw to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package omw-1.4 to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package opinion_lexicon to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\opinion_lexicon.zip.
[nltk_data]    | Downloading package panlex_swadesh to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data]    | Downloading package paradigms to
[nltk_data]    |     C:\Users\abybi\AppData\Roaming\nltk_data...
[nltk_data

In [ ]:
stemmer = PorterStemmer()
tokenizer = word_tokenize
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_html(text):
    soup = BeautifulSoup(''.join(text), 'html.parser')
    return soup.get_text()

def preprocess(text):
    text = clean_html(text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    tokens = tokenizer(text)
    lemma = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(lemma)

In [ ]:
df['review'] = df['review'].apply(preprocess)

In [ ]:
df['review_vectors'] = df['review'].apply(lambda x: map_words_to_vector(x,wv_from_bin))

In [ ]:
print(df.review_vectors[1].shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['review_vectors'].values, df['sentiment'].values, test_size=0.2, random_state=2)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(verbose = 1, C = 0.01)
# Pad the arrays in X_train
max_len = 225
X_train_padded = np.zeros((len(X_train), max_len, len(X_train[0][0])))
for i, x in enumerate(X_train):
    if len(x) > 0:
        X_train_padded[i, :min(len(x), max_len), :] = x[:min(len(x), max_len), :]
X_train_stacked = np.reshape(X_train_padded, (len(X_train), -1))

In [ ]:
X_train_stacked.shape

In [ ]:
model.fit(X_train_stacked, y_train)

In [ ]:
max_len = 225
X_test_padded = np.zeros((len(X_test), max_len, len(X_test[0][0])))
for i, x in enumerate(X_test):
    if len(x) > 0:
        X_test_padded[i, :min(len(x), max_len), :] = x[:min(len(x), max_len), :]
X_test_stacked = np.reshape(X_test_padded, (len(X_test), -1))
from sklearn.metrics import accuracy_score
print(accuracy_score(model.predict(X_test_stacked), y_test))

In [ ]:
model = model = LogisticRegression(verbose = 1, C = 1, max_iter = 200)
model.fit(X_train_stacked, y_train)
print(accuracy_score(model.predict(X_test_stacked), y_test))

In [ ]:
from sklearn.svm import LinearSVC
lsvc = LinearSVC()

In [ ]:
lsvc.fit(X_train_stacked, y_train)
predict_lsvc = lsvc.predict(X_test_stacked)
accuracy_lsvc = accuracy_score(predict_lsvc, y_test)
print(accuracy_lsvc)